# Sentiment analysis notebook

In [1]:
import pandas as pd
import numpy as np

In [2]:
sentiment_lex = pd.read_csv('../dataset/sensaldo-fullform-v02.txt',
                            skiprows=58)
df_full = pd.read_csv("../dataset/lawline_data.csv")

## Clean sentiment DF

In [3]:
df_list = []
for item in sentiment_lex.values:
    row_list = item[0].split('\t')

    word = row_list[0]
    sent = row_list[2].split(':')[1]
    if len(sent) > 1:
        if sent[0] == '-':
            sent = sent[:2]
        else:
            sent = sent[0]
    
    df_list.append([word, float(sent)])

In [4]:
df_sent = pd.DataFrame(df_list, columns=['word', 'sentiment'])

## Clean text data

First, we make texts lowercase and remove extra whitespaces, newlines, etc.

In [5]:
df_full["text_clean"] = df_full["text"].str.lower()
df_full["text_clean"] = [" ".join(str(item).split()) for item in df_full["text_clean"]]

Next we remove punctuation, numbers and remove excessive spaces between words.

In [6]:
df_full["text_clean"] = df_full["text_clean"].str.replace("[^\w\s]", "")
df_full["text_clean"] = df_full["text_clean"].str.replace("[0-9]", "")
df_full["text_clean"] = df_full["text_clean"].str.replace("\s{2,}", " ")

C:\Users\samue\AppData\Local\Temp\ipykernel_17112\1199548694.py:1: FutureWarning: The default value of regex will change from True to False in a future version.
  df_full["text_clean"] = df_full["text_clean"].str.replace("[^\w\s]", "")
C:\Users\samue\AppData\Local\Temp\ipykernel_17112\1199548694.py:2: FutureWarning: The default value of regex will change from True to False in a future version.
  df_full["text_clean"] = df_full["text_clean"].str.replace("[0-9]", "")
C:\Users\samue\AppData\Local\Temp\ipykernel_17112\1199548694.py:3: FutureWarning: The default value of regex will change from True to False in a future version.
  df_full["text_clean"] = df_full["text_clean"].str.replace("\s{2,}", " ")


Make each text a list of words. Remove all words not included in sentiment dictionary

In [7]:
list_texts = df_full["text_clean"].values.tolist()

In [12]:
list_texts = [[word for word in text.split(' ') if word in df_sent['word'].values.tolist()] for text in list_texts]

In [14]:
def match_word(word):
    idx = df_sent.index[df_sent['word'] == word]
    if len(idx) == 1:
        sent = df_sent['sentiment'].iloc[idx[0]]
        return sent
    return np.nan

def match_list(text):
    temp = list(map(match_word, text))

    if len(temp) > 0:
        sent_text = np.nanmean(temp)
    else:
        sent_text = 0

    return sent_text

In [30]:
sent_list = list(map(match_list, list_texts))

C:\Users\samue\AppData\Local\Temp\ipykernel_17112\323735224.py:12: RuntimeWarning: Mean of empty slice
  sent_text = np.nanmean(temp)


In [32]:
df_full['sentiment'] = sent_list

In [33]:
df_full.to_csv('../dataset/df_with_sent.csv', index=False)